# Black-box Attack API

In [ ]:
!git clone https://github.com/Yueeeeeeee/RecSys-Extraction-Attack.git

Cloning into 'RecSys-Extraction-Attack'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 77 (delta 25), reused 66 (delta 19), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [ ]:
%cd RecSys-Extraction-Attack/

/content/RecSys-Extraction-Attack


In [ ]:
!apt-get install libarchive-dev
!pip install faiss-cpu --no-cache
!apt-get install libomp-dev
!pip install wget
!pip install libarchive

In [ ]:
def zero_gradients(x):
    if isinstance(x, torch.Tensor):
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
    elif isinstance(x, collections.abc.Iterable):
        for elem in x:
            zero_gradients(elem)

## Black-Box Model Training

**NARM model trained on ML-1M dataset.**

Given a user sequence 𝒙 with length 𝑇 , we use $𝒙_{[:𝑇−2]}$ as training data and use the last two items for validation and testing respectively. We use hyper-parameters from grid-search. Additionally, all models are trained using Adam optimizer with weight decay 0.01, learning rate 0.001, batch size 128 and 100 linear warmup steps, allowed sequence length as 200.

We accelerate evaluation by uniformly sampling 100 negative items for each user. Then we rank them with the positive item and report the average performance on these 101 testing items. Our Evaluation focuses on two aspects:
- Ranking Performance: We to use truncated Recall@K that is equivalent to Hit Rate (HR@K) in our evaluation, and Normalized Discounted Cumulative Gain (NDCG@K) to measure the ranking quality.
- Agreement Measure: We define Agreement@K (Agr@K) to evaluate the output similarity between the black-box model and our extracted white-box model.

Official results:

<img src='_images/T697871_1.png'>

In [ ]:
!python train.py

Input 1 / 20 for movielens, b for beauty, bd for dense beauty, g for games, s for steam and y for yoochoose: 1
Input model code, b for BERT, s for SASRec and n for NARM: n
Input GPU ID: 0
Raw file doesn't exist. Downloading...

Extracting data...

/content/RecSys-Extraction-Attack/datasets/ml_1m.py:87: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='::', header=None)
Filtering triplets
Densifying index
Splitting
100% 6040/6040 [00:08<00:00, 685.31it/s]
Negative samples don't exist. Generating.
Sampling negative items randomly...
100% 6040/6040 [00:01<00:00, 5988.52it/s]
Negative samples don't exist. Generating.
Sampling negative items randomly...
100% 6040/6040 [00:01<00:00, 5951.72it/s]
Failed to load old model, continue training new model...
Eval: N@1 0.008, N@5

In [ ]:
# !zip -r bb_model_narm_ml1m.zip ./experiments
# !cp bb_model_narm_ml1m.zip /content/drive/MyDrive/TempData
# !ls /content/drive/MyDrive/TempData

## White-Box Model Distillation

In [ ]:
!cp /content/drive/MyDrive/TempData/bb_model_narm_ml1m.zip .
!unzip bb_model_narm_ml1m.zip

Archive:  bb_model_narm_ml1m.zip
   creating: experiments/
   creating: experiments/narm/
   creating: experiments/narm/ml-1m/
   creating: experiments/narm/ml-1m/logs/
  inflating: experiments/narm/ml-1m/logs/events.out.tfevents.1631975614.0dbd3d97f40f.706.0  
   creating: experiments/narm/ml-1m/models/
  inflating: experiments/narm/ml-1m/models/best_acc_model.pth  
  inflating: experiments/narm/ml-1m/models/checkpoint-recent.pth  


In [ ]:
!python distill.py

Input 1 / 20 for movielens, b for beauty, bd for dense beauty, g for games, s for steam and y for yoochoose: 1
Input model code, b for BERT, s for SASRec and n for NARM: n
Input GPU ID: 0
Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.
Input black box model code, b for BERT, s for SASRec and n for NARM: n
Input integer number of seqs budget: 4
Generating dataset...
100% 60/60 [01:40<00:00,  1.68s/it]
## Distilling model via autoregressive data... ##
Eval: N@1 0.007, N@5 0.027, N@10 0.043, R@1 0.007, R@5 0.048, R@10 0.100: 100% 24/24 [00:00<00:00, 24.84it/s]
Update Best NDCG@10 Model at 1
  0% 0/4665 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:151: UserWarning: sample_n will be deprecated. Use .sample((n,)) instead
  warnings.warn('sample_n will be deprecated. Use .sample((n,)) instead', UserWarning)
Epoch 1 Stage 1, loss 0.548 : 100% 4665/4665 [07:58<00:00,  9.75it/s]
Eval: N@1 0

In [ ]:
!zip -r wb_model_narm_ml1m.zip ./experiments
!cp wb_model_narm_ml1m.zip /content/drive/MyDrive/TempData
!ls /content/drive/MyDrive/TempData

## Attack

In [ ]:
!python attack.py

Input 1 / 20 for movielens, b for beauty, bd for dense beauty, g for games, s for steam and y for yoochoose: 1
Input model code, b for BERT, s for SASRec and n for NARM: n
Input GPU ID: 0
Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.
Input white box model code, b for BERT, s for SASRec and n for NARM: n
{1: 'narm2narm_autoregressive4', 2: 'narm_black_box'}
Input index of desired white box model: 1
Already preprocessed. Skip preprocessing
## Black-Box Targeted Test on Item 2459 ##
Val: R@1 0.159, R@5 0.525, R@10 0.732, N@5 0.345, N@10 0.412: 100% 48/48 [00:01<00:00, 24.67it/s]
## Targeted Attack on Item 2459 ##
  0% 0/48 [00:00<?, ?it/s]/content/RecSys-Extraction-Attack/adversarial/attacker.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  perturbed_seqs = torch.cat((pertu

In [ ]:
!zip -r wb_model_narm_ml1m.zip ./experiments
!cp wb_model_narm_ml1m.zip /content/drive/MyDrive/TempData
!ls /content/drive/MyDrive/TempData

## Retrain

In [ ]:
!python retrain.py

Input 1 / 20 for movielens, b for beauty, bd for dense beauty, g for games, s for steam and y for yoochoose: 1
Input model code, b for BERT, s for SASRec and n for NARM: n
Input GPU ID: 0
Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.
Input white box model code, b for BERT, s for SASRec and n for NARM: n
{1: 'narm2narm_autoregressive4', 2: 'narm_black_box'}
Input index of desired white box model: 1
Already preprocessed. Skip preprocessing
## Generate Biased Data with Target [2459, 1009, 2135, 918, 3233, 1226, 498, 2917, 1332, 3184, 264, 2490, 1696, 1448, 144, 365, 1368, 2714, 1874, 3285, 2235, 3406, 3155, 1322, 2928] ##
Generating poisoned dataset...
100% 2/2 [00:04<00:00,  2.24s/it]
Already preprocessed. Skip preprocessing
Negative samples don't exist. Generating.
Sampling negative items randomly...
100% 6100/6100 [00:01<00:00, 6058.91it/s]
Negative samples don't exist. Generating.
Sampling negative items randomly...
100% 6